In [1]:
import csv

# Define the file path
file_path = 'Students.csv'

# Initialize department counters
department_counts = {'CS': 0, 'EE': 0, 'AI': 0, 'BBA': 0}

# Read the CSV file
with open(file_path, 'r') as file:
    reader = csv.DictReader(file)
    
    # Iterate over each row in the CSV file
    for row in reader:
        # Extract the department from the row
        department = row['Dept']
        
        # Increment the count for the department
        if department in department_counts:
            department_counts[department] += 1

# Print the department counts
for department, count in department_counts.items():
    print(f"{department}: {count}")

CS: 704
EE: 682
AI: 660
BBA: 660


In [3]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder

# Ask the user to enter the students CSV file name
students_file_name = 'Students.csv'

# Ask the user to enter the teachers CSV file name
teachers_file_name = 'Teachers.csv'

# Load the data
students_df = pd.read_csv(students_file_name)
teachers_df = pd.read_csv(teachers_file_name)

# Define the number of rooms and their capacities
num_rooms = 30
room_capacities = [30] * 25 + [25] * 5  # 25 rooms with 30 seats, 5 rooms with 25 seats

# Define the exam schedule
exam_schedule = {
    'Monday': 'CS',
    'Tuesday': 'EE',
    'Wednesday': 'AI',
    'Thursday': 'BBA'
}

# Perform one-hot encoding on the categorical features
encoder = OneHotEncoder()
X = encoder.fit_transform(students_df[['Batch', 'Dept']]).toarray()

# Perform k-means clustering
kmeans = KMeans(n_clusters=len(exam_schedule), n_init=10, random_state=42)
students_df['Cluster'] = kmeans.fit_predict(X)

# Generate the seating plan and faculty allocation
exam_sitting_plan = []
for day, dept in exam_schedule.items():
    students = students_df[students_df['Dept'] == dept]
    clusters = students['Cluster'].unique()
    for cluster in clusters:
        cluster_students = students[students['Cluster'] == cluster]
        room_num = 1
        for i in range(0, len(cluster_students), room_capacities[0]):
            batch_students = cluster_students.iloc[i:i+room_capacities[0]]
            faculty = teachers_df[teachers_df['Dept'] == dept].sample(1)
            for _, student in batch_students.iterrows():
                exam_sitting_plan.append({
                    'Room': f'Room {room_num}',
                    'Cluster': cluster,
                    'Batch': student['Batch'],
                    'Department': student['Dept'],
                    'Exam Day': day,
                    'Student Name': student['Name'],
                    'Student ID': student['ID'],
                    'Faculty': faculty['Name'].values[0]
                })
            room_num += 1

# Save the report to a CSV file
exam_sitting_plan_df = pd.DataFrame(exam_sitting_plan)
exam_sitting_plan_df.to_csv('exam_sitting_plan.csv', index=False)
